# Creating the rule set

In [1]:
from os import path
from typing import *
import re
import subprocess
import sys

sys.path.insert(0, "../..")
from copy_files import copy_files, languagetool_path
from shared import add_to_dict, csvs_to_dict, dict_to_csvs, log
from xmlify import rule_to_xml

In [2]:
unified_dic: Dict[str, Dict[str, List[str]]] = {"sg": {}, "pl": {}, "combined": {}}

files = {
    "dereko/dereko_unified": ["sg", "pl"],
    "geschicktgendern/geschicktgendern": ["sg", "pl"],
    "openthesaurus/openthesaurus_persons_male": ["sg", "pl"],
    "vienna_catalog/vienna_catalog": ["sg", "pl"],
}
dic = {}

for file, numbers in files.items():
    dic[file] = csvs_to_dict(file, numbers=numbers)
    for n in numbers:
        print(file, n, len(dic[file][n]))
        for key, vals in dic[file][n].items():
            add_to_dict(key, vals, unified_dic[n])
            add_to_dict(key, vals, unified_dic["combined"])


sorted(list(unified_dic["sg"].items()))[:10]

dereko/dereko_unified sg 86
dereko/dereko_unified pl 2748
geschicktgendern/geschicktgendern sg 1185
geschicktgendern/geschicktgendern pl 1241
openthesaurus/openthesaurus_persons_male sg 3490
openthesaurus/openthesaurus_persons_male pl 3490
vienna_catalog/vienna_catalog sg 975
vienna_catalog/vienna_catalog pl 1004


[('AHS-Lehrer', ['AHS-Lehrerin bzw. AHS-Lehrer']),
 ('Aasgeier', []),
 ('Abbrecherquote', ['Abbruchquote']),
 ('Abdecker', []),
 ('Abenteurer',
  ['Waghals',
   'abenteuerliebende Person',
   'abenteuerlustige Person',
   'abenteuermutige Person',
   'Abenteuermensch']),
 ('Abfallmanager', ['Abfallmanagerin und Abfallmanager']),
 ('Abgeordneter', []),
 ('Abgänger', ['absolvierende Person', 'Abschluss innehabende Person']),
 ('Abiturient', ['Abitur ablegende Person', 'Person, die Abitur macht']),
 ('Abkömmling',
  ['abstammende Person',
   'nachkommende Person',
   'Kind',
   'Kindeskind',
   'Person gleicher Abstammung'])]

## Custom rules

We add some custom rules that we have written ourselves, inspired in part by the _retext-equality_ data set. 

In [3]:
custom_xml = open(path.join("retext-equality", "custom_rules_disability.xml")).read()

## Conversion to proper LanguageTool XML format

The LanguageTool rule format is described [over here](https://web.archive.org/web/20210910183442/https://dev.languagetool.org/development-overview) and [here](https://dev.languagetool.org/tips-and-tricks).

We devise a function to convert a _geschickt gendern_ entry to a XML LanguageTool entry.

In [4]:
unified_dic["pl"]["Angreifer"]

['Angreifer*innen', 'Angreifende']

In [5]:
print(rule_to_xml("Angreifer", "pl", unified_dic["pl"]["Angreifer"]))


<rule id="Angreifer_pl" name="Angreifer">
    <antipattern><token>Angreifer</token><token>*</token><token>innen</token></antipattern>
		<antipattern><token>Angreifende</token></antipattern>
    <pattern><token inflected="yes" postag=".*:PLU:.*" postag_regexp="yes"><exception postag=".*:SIN:.*" postag_regexp="yes" />Angreifer</token></pattern>
    <suggestion>Angreifer*innen</suggestion>, <suggestion>Angreifende</suggestion>
    <message>Der Stern wird in den letzten Jahren zunehmend verwendet. Besonders häufig findet man das Sternchen im Kontexten, in denen aufgrund aktuelle Transgender- und Intersexualitätsdebatten nicht von lediglich zwei Geschlechtern ausgegangen wird, Geschlecht also nicht mehr als ein binäre System verstanden wird. Mit dem Sternchen soll bewusst irritiert und die Möglichkeit weitere Kategorien angedeutet werden.</message>
    <short>Die Bezeichnung "Angreifer" spricht nur männliche Leser an. Versuche alle Menschen anzusprechen.</short>
    <example correction="An

In [6]:
print(rule_to_xml("Angreifer", "both", unified_dic["combined"]["Angreifer"]))


<rule id="Angreifer_both" name="Angreifer">
    <antipattern><token>Angreifer</token><token>*</token><token>innen</token></antipattern>
		<antipattern><token>angreifende</token><token>Person</token></antipattern>
		<antipattern><token>Angreifende</token></antipattern>
    <pattern><and>
	<token inflected="yes" postag=".*:SIN:.*" postag_regexp="yes">Angreifer</token>
	<token inflected="yes" postag=".*:PLU:.*" postag_regexp="yes">Angreifer</token>
</and></pattern>
    <suggestion>Angreifer*innen</suggestion>, <suggestion>angreifende Person</suggestion>, <suggestion>Angreifende</suggestion>
    <message>Der Stern wird in den letzten Jahren zunehmend verwendet. Besonders häufig findet man das Sternchen im Kontexten, in denen aufgrund aktuelle Transgender- und Intersexualitätsdebatten nicht von lediglich zwei Geschlechtern ausgegangen wird, Geschlecht also nicht mehr als ein binäre System verstanden wird. Mit dem Sternchen soll bewusst irritiert und die Möglichkeit weitere Kategorien anged

In [7]:
xml = custom_xml
for key, val in unified_dic["combined"].items():
    for number in ["sg", "pl"]:
        if key in unified_dic[number].keys():
            xml += rule_to_xml(key, number, unified_dic[number][key])
    for number in ["both", "unknown"]:
        xml += rule_to_xml(key, number, val)
open("grammar_openminded.xml", "w").write(xml)

26376775

## Injecting the rules to the existing LanguageTool rule file

In [ ]:
copy_files()

## Validating and using the rules

Running the LanguageTool rule validation:

In [ ]:
# result = subprocess.run(["./testrules.sh", "de"], cwd=languagetool_path, capture_output=True)
# stdout = result.stdout.decode("UTF-8")
# stderr = result.stderr.decode("UTF-8")
# open("rule_validation_stdout.log", "w").write(stdout)
# open("rule_validation_stderr.log", "w").write(stderr)

Starting LanguageTool:

In [ ]:
# subprocess.run(["java", "-jar", path.join(languagetool_path, "languagetool.jar")])